In [53]:
import pandas as pd
import ruamel.yaml as yaml
from copy import deepcopy as copy
from pprint import pprint
import os

In [33]:
data = pd.read_csv("gaussian_camel_integrands.csv")
data["d"] = data["d"].astype(int)

In [47]:
base = yaml.load(open("../benchmarks/benchmarks_01/camel/camel_grid_config.yaml",'r'),Loader=yaml.Loader)
del base["integrator_config_grid"]["n_points_survey"]
base["integrator_config_grid"]["n_hidden"] = [2,4,8,16]
base["integrator_config_grid"]["loss"] = ["variance", "dkl"]
base["experiment_name"] = "camel_scan_per_dim"

In [48]:
pprint(base)

{'base_integrator_config': {'checkpoint_on_cuda': False,
                            'masking': 'iflow',
                            'minibatch_size': 20000,
                            'n_bins': 50,
                            'n_epochs': 50,
                            'n_iter': 10,
                            'n_points_survey': 100000,
                            'repetitions': 1},
 'dbname': 'benchmarks_01/benchmarks.db',
 'debug': False,
 'dimensions': [2, 4, 8, 16, 32],
 'experiment_name': 'camel_scan_per_dim',
 'integrand_params_grid': {'s': [0.1, 0.3, 0.5, 0.7]},
 'integrator_config_grid': {'d_hidden': [64, 128, 256],
                            'flow': ['pwlinear', 'pwquad'],
                            'loss': ['variance', 'dkl'],
                            'lr': [0.005, 0.001, 0.0005, 0.0001, 5e-05, 1e-05],
                            'masking': ['iflow', 'checkerboard'],
                            'n_bins': [2, 10, 50],
                            'n_epochs': [2, 10, 50],

In [49]:
dimensions = data["d"].unique()
sigma_camels = data.set_index(["d"])["sigma_camel"]

In [50]:
camel_configs = dict()
for d in dimensions:
    camel_configs[d] = copy(base)
    camel_configs[d]["dimensions"] = [int(d)]
    camel_configs[d]["integrand_params_grid"] = {'s': sigma_camels[d].to_list()}

In [51]:
for d, config in camel_configs.items():
    out_file = open(f"../benchmarks/benchmarks_01/camel/camel_{d}d_grid_config.yaml",'w+')
    yaml.dump(config,out_file,Dumper=yaml.Dumper)
    out_file.close()

In [54]:
dimensions = data["d"].unique()
base["experiment_name"] = "gaussian_scan_per_dim"
sigma_gauss = data.set_index(["d"])["sigma_gauss"]

camel_configs = dict()
for d in dimensions:
    camel_configs[d] = copy(base)
    camel_configs[d]["dimensions"] = [int(d)]
    camel_configs[d]["integrand_params_grid"] = {'s': sigma_gauss[d].to_list()}
    
for d, config in camel_configs.items():
    filename = f"../benchmarks/benchmarks_01/gauss/gauss_{d}d_grid_config.yaml"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    out_file = open(filename,'w+')
    yaml.dump(config,out_file,Dumper=yaml.Dumper)
    out_file.close()